In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords

In [ ]:
stop = stopwords.words('english')

In [ ]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
    + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [ ]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [ ]:
next(stream_docs(path='movie_data.csv'))

In [ ]:
def get_minibatch(doc_steam, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_steam)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                        n_features=2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)

In [ ]:
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)

In [ ]:
doc_stream = stream_docs(path='movie_data.csv')

In [ ]:
import pyprind
pbar = pyprind.ProgBar(45)

In [ ]:
classes = np.array([0, 1])

In [ ]:
for i in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

In [ ]:
X_test, y_test = get_minibatch(doc_stream, 5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

In [ ]:
import pickle
import os

In [ ]:
dest = os.path.join('movieclassifier', 'pkl_object')

In [ ]:
if not os.path.exists(dest):
    os.makedirs(dest)

In [ ]:
pickle.dump(stop, 
           open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
           protocol=4)

In [ ]:
pickle.dump(clf,
           open(os.path.join(dest, 'classifier.pkl'), 'wb'),
           protocol=4)